## 0.0 setup

1. follow the setup instructions in `1_0_hybridization_setup.ipynb`

2. create directories and copy database files from the shared drive (`/srv/data/autumn_school/hybridization/`)

In [1]:
!rm -rf ~/hybridization_data/databases_hybrid_from_shared_drive 
!mkdir -p ~/hybridization_data/databases_hybrid_from_shared_drive
!mkdir -p ~/hybridization_data/databases_hybrid
!cp -a /srv/data/autumn_school/hybridization/databases_hybrid/* ~/hybridization_data/databases_hybrid_from_shared_drive

## 0.1. imports
### 0.1.1. regular imports

In [2]:
# i/o
import sys
import os
from pathlib import Path
import gzip
import pickle
# configuration
import yaml
# lca
import brightway2 as bw
import ecospold2matrix as e2m
import pymrio
# data science
import pandas as pd

Using environment variable BRIGHTWAY2_DIR for data directory:
/home/weinold/bw_data


### 0.1.2. local imports

In [3]:
sys.path.append(os.path.join(Path.home(), 'pylcaio', 'src')) # required for local import of pylcaio
import pylcaio

## 0.2. file paths
### 0.2.1. directories

In [4]:
%%capture
print(path_dir_databases_hybrid := os.path.join(Path.home(), 'hybridization_data/databases_hybrid'))
print(path_dir_databases_hybrid_from_shared_drive := os.path.join(Path.home(), 'hybridization_data/databases_hybrid_from_shared_drive'))

### 0.2.2. databases

In [5]:
%%capture
print(path_file_hybrid := os.path.join(path_dir_databases_hybrid, 'hybrid_ecoinvent_3_5_exiobase_monetary_pxp_2012.pickle'))
print(path_file_hybrid_from_shared_drive := os.path.join(path_dir_databases_hybrid_from_shared_drive, 'pylcaio_class_instance_after_hybrid.pkl'))

## 1.1. import hybrid database

⚠️ if you would like to use the hybrid database from the shared drive (`/srv/data/autumn_school/`), set 

```
load_from_shared_drive: bool = True
```

⚠️ if you would like to use the hybrid databased created (by yourself) in the notebook `1_1_hybridiation_tutorial_hybridization.ipynb`, set

```
load_from_shared_drive: bool = False
```

In [6]:
load_from_shared_drive: bool = False

In [8]:
if load_from_shared_drive == False:
    with open(path_file_hybrid, 'rb') as filestream:
        lcaio_object_after_hybridization: pymrio.core.mriosystem.IOSystem = pd.read_pickle(filestream)
if load_from_shared_drive == True:
    with open(path_file_hybrid_from_shared_drive, 'rb') as filestream:
        lcaio_object_after_hybridization: pymrio.core.mriosystem.IOSystem = pd.read_pickle(filestream)

UnpicklingError: invalid load key, '\x1f'.

## 2.1. analysis

In [10]:
Analysis = pylcaio.Analysis(path_file_hybrid)

KeyError: 'method'